In [ ]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

from time import perf_counter, sleep
import itertools, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
generatorPath

import tensorflow as tf
import tensorflow_hub as hub

print('hub: ', hub.__version__)
url="https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5"
print(url)

In [ ]:
data = DataLoader.from_folder(generatorPath)
train_ds, restData = data.split(0.9)
val_ds, test_ds = restData.split(0.5)
print()
print('\nnum_classes:', train_ds.num_classes)
print('class names:', train_ds.index_to_label)
print(len(train_ds) // 8, len(val_ds)//8)

In [ ]:
from tflite_model_maker import image_classifier
BATCH_SIZE = 8
EPOCHS = 2
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.001

model = image_classifier.create(
    train_data = train_ds,
    validation_data = val_ds,
    model_spec = model_spec.get('mobilenet_v2'),
    learning_rate = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    dropout_rate = DROPOUT_RATE,
    use_augmentation = True, # default=False
    use_hub_library = True,  # default=True
    momentum = 0.9, # Only used when use_hub_library is True
    shuffle = False,
    train_whole_model = False
)

In [ ]:
loss, accuracy = model.evaluate(test_ds)

loss = round(loss, 4)
accuracy = round(accuracy, 1)
print(f'loss: {loss}\taccuracy: {accuracy * 100}%')